In [ ]:
%%capture
!pip install duckdb spatialpandas

In [ ]:
import ngen
import os
import duckdb
import sys

import holoviews as hv
import geoviews as gv
import geopandas as gpd
import panel as pn
import hvplot.pandas #noqa
import pandas as pd
import dask.dataframe as dd
import cartopy.crs as ccrs

from pathlib import Path

In [ ]:
# add evaluation to path.  ultimately this will be installed.
sys.path.insert(0, '../../evaluation')
import utils

In [ ]:
# Set some configurations
NGEN_DIR = "/home/jovyan/cache/AWI_03W_113060_001_NGEN/"
NGEN_CONFIG_DIR = Path(NGEN_DIR, "config")
NGEN_FORCINGS_DIR = Path(NGEN_DIR, "forcings")
NGEN_OUTPUT_DIR = Path(NGEN_DIR, "output")

STUDY_DIR = "/home/jovyan/cache/AWI_03W_113060_001_CHEET/"
STUDY_FORCINGS_DIR = Path(STUDY_DIR, "forcings")
STUDY_OUTPUT_DIR = Path(STUDY_DIR, "output")
STUDY_GEO_DIR = Path(STUDY_DIR, "geo")
STUDY_USGS_DIR = Path(STUDY_DIR, "usgs")

In [ ]:
# Load up the NextGen geospatial data

In [ ]:
catchment_file_gdf = gpd.read_file(Path(NGEN_CONFIG_DIR, "catchment_data.geojson")).to_crs("EPSG:3857")
catchment_file_gdf 

In [ ]:
catchment_polygons = catchment_file_gdf.hvplot(color="blue", crs=ccrs.GOOGLE_MERCATOR) #, hover_cols=["id", "toid"])

In [ ]:
nexus_file_gdf = gpd.read_file(Path(NGEN_CONFIG_DIR, "nexus_data.geojson")).to_crs("EPSG:3857")
nexus_file_gdf 

In [ ]:
nexus_points = nexus_file_gdf.hvplot(color="green", crs=ccrs.GOOGLE_MERCATOR, hover_cols=["id", "toid"])

In [ ]:
tiles = gv.tile_sources.OSM

In [ ]:
tiles * catchment_polygons * nexus_points 

In [ ]:
# gdf = gdf = gpd.read_file(Path(STUDY_GEO_DIR, "nextgen_03W.gpkg"))

In [ ]:
# gdf.hvplot()

In [ ]:
# Query NextGen timeseries data

In [ ]:
query = f"""
    SELECT catchment_id, value_time, value
    FROM read_parquet('{STUDY_OUTPUT_DIR}/cat-*.parquet')
    WHERE catchment_id = 'cat-113052';
"""
df = duckdb.query(query).to_df()
df.hvplot()

In [ ]:
query = f"""
    SELECT catchment_id, value_time, value
     FROM read_parquet('{STUDY_FORCINGS_DIR}/cat-*.parquet')
    WHERE catchment_id = 'cat-113052';
"""
df = duckdb.query(query).to_df()
df.hvplot()

In [ ]:
usgs_gdf = utils.get_usgs_gages().to_crs("EPSG:3857")
usgs_gdf

In [ ]:
usgs = usgs_gdf.hvplot(color="red", crs=ccrs.GOOGLE_MERCATOR, hover_cols=["gage_id"])

In [ ]:
tiles * catchment_polygons * nexus_points * usgs

In [ ]:
layout = pn.Column(tiles * catchment_polygons * nexus_points * usgs).servable()